# Notebook for Coursera Capstone project

In [1]:
## Import libs
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library

In [3]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!
